In [1]:
import pandera as pa
from pandera import Column, DataFrameSchema, Check
import re
import duckdb
import pandas as pd

# Caminho para o banco de dados DuckDB
db_path = "D:\\GitHub\\projeto_Analytics_Engineering\\data_quality\\silver_1.duckdb"

In [6]:
# Função para validar o formato monetário como uma string
def check_price_format(val):
    return bool(re.match(r"^\$\d+(,\d{3})*(\.\d{2})?$", val))

# Definindo o esquema para a tabela "listings"
listings_schema = DataFrameSchema({
    "id": Column(pa.Int64, checks=Check.greater_than(0), nullable=False),  # ID não nulo e maior que 0
    "neighbourhood_cleansed": Column(pa.String, nullable=False),           # Bairro não nulo
    #"price": Column(pa.String, checks=Check(check_price_format, element_wise=True), nullable=False),  # Preço obrigatório, formato monetário
    "number_of_reviews": Column(pa.Int64, checks=Check.greater_than_or_equal_to(0), nullable=False),   # Número de avaliações, não pode ser nulo, mínimo de 0
    #"review_scores_rating": Column(pa.Float64, checks=Check.in_range(0, 5), nullable=False)          # Pontuação entre 0 e 5, não pode ser nulo
})


In [ ]:
# Conectando ao banco de dados DuckDB
conn = duckdb.connect(database=db_path, read_only=False)

# Carregando os dados da tabela "listings"
df_listings = conn.execute("SELECT distinct * FROM listings").fetchdf()

# Validando o DataFrame com Pandera
try:
    validated_listings_df = listings_schema.validate(df_listings)
    print("Dados validados com sucesso:")
    print(validated_listings_df)
except pa.errors.SchemaError as e:
    print("Erro na validação dos dados:", e)
finally:
    conn.close()  # Fechando a conexão com o banco de dados

Dados validados com sucesso:
                        id                                       listing_url  \
0                   230989               https://www.airbnb.com/rooms/230989   
1                   231105               https://www.airbnb.com/rooms/231105   
2                   231497               https://www.airbnb.com/rooms/231497   
3                    48901                https://www.airbnb.com/rooms/48901   
4                   236991               https://www.airbnb.com/rooms/236991   
...                    ...                                               ...   
34659  1186981497123032289  https://www.airbnb.com/rooms/1186981497123032289   
34660  1187078788620039832  https://www.airbnb.com/rooms/1187078788620039832   
34661  1184700608791969952  https://www.airbnb.com/rooms/1184700608791969952   
34662  1184818035513976907  https://www.airbnb.com/rooms/1184818035513976907   
34663  1184844634544738969  https://www.airbnb.com/rooms/1184844634544738969   

          

## Resumo:

coluna price: 972 nulos

coluna review_scores_rating: 8446 nulos